In [3]:
import os
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives import hashes as crypto_hashes
from cryptography.hazmat.primitives import hashes


class GTTSecured:
    def __init__(self, password: str):
        """
        Initialize with a password to derive the encryption key.
        :param password: The password used to derive the encryption key.
        """
        self.password = password
        self.salt = os.urandom(16)  # Random salt for key derivation
        self.key = self.derive_key(password, self.salt)  # Generate the AES key

    def derive_key(self, password: str, salt: bytes) -> bytes:
        """
        Derives a secure encryption key from the password and salt using PBKDF2.
        :param password: The password used for key derivation.
        :param salt: A random salt to be used with the password.
        :return: The derived AES encryption key (32 bytes for AES-256).
        """
        kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256(),
            length=32,  # Length of the key (32 bytes = 256-bit key for AES-256)
            salt=salt,
            iterations=100000,  # Increase iterations for higher security
            backend=default_backend()
        )
        return kdf.derive(password.encode())

    def encrypt(self, plaintext: str) -> str:
        """
        Encrypts the provided plaintext using AES encryption (GCM mode).
        :param plaintext: The plaintext to encrypt.
        :return: The encrypted data (ciphertext) as a base64 encoded string.
        """
        # Generate a random IV (Initialization Vector) for GCM mode
        iv = os.urandom(12)

        # Pad plaintext to be multiple of block size (AES block size is 128 bits = 16 bytes)
        padder = padding.PKCS7(128).padder()
        padded_data = padder.update(plaintext.encode()) + padder.finalize()

        # Set up the AES GCM cipher with the derived key, IV, and tag length
        cipher = Cipher(algorithms.AES(self.key), modes.GCM(iv), backend=default_backend())
        encryptor = cipher.encryptor()

        # Perform the encryption and get the tag
        ciphertext = encryptor.update(padded_data) + encryptor.finalize()
        tag = encryptor.tag

        # Return as a base64-encoded string for easy storage/transfer
        encrypted_data = iv + tag + ciphertext  # Combine IV, tag, and ciphertext

        return encrypted_data.hex()  # Convert to hex string for easy storage/transfer

    def decrypt(self, encrypted_data_hex: str) -> str:
        """
        Decrypts the provided ciphertext using AES encryption (GCM mode).
        :param encrypted_data_hex: The encrypted data (ciphertext) in hex format.
        :return: The decrypted plaintext.
        """
        # Convert the hex string back to bytes
        encrypted_data = bytes.fromhex(encrypted_data_hex)

        # Extract the IV, tag, and ciphertext
        iv = encrypted_data[:12]  # The first 12 bytes are the IV
        tag = encrypted_data[12:28]  # The next 16 bytes are the authentication tag
        ciphertext = encrypted_data[28:]  # The remaining part is the ciphertext

        # Set up the AES GCM cipher for decryption with the same key and IV
        cipher = Cipher(algorithms.AES(self.key), modes.GCM(iv, tag), backend=default_backend())
        decryptor = cipher.decryptor()

        # Perform decryption and remove padding
        decrypted_data = decryptor.update(ciphertext) + decryptor.finalize()

        # Unpad the decrypted data
        unpadder = padding.PKCS7(128).unpadder()
        plaintext = unpadder.update(decrypted_data) + unpadder.finalize()

        return plaintext.decode()  # Return as a string

        # Initialize the GTTSecured class with a password
password = "Concussion@2025"
gtt_secured = GTTSecured(password)

# Encrypt a string (e.g., a password or any other data)
plaintext = 'e=R~pf"/r2OPvA2e9^87'
encrypted_data = gtt_secured.encrypt(plaintext)
print(f"Encrypted: {encrypted_data}")

# Decrypt the encrypted data back to the original plaintext
decrypted_data = gtt_secured.decrypt(encrypted_data)
print(f"Decrypted: {decrypted_data}")

Encrypted: 417680244602dee0fc5b4753d3976db7bd0c878432139eb3ee318b9af1725de9232df08d1df4e15d7c046381d49f0fd2ace71cf01d02f61cb8777884
Decrypted: e=R~pf"/r2OPvA2e9^87
